In [112]:
import torch
import pandas as pd
import numpy as np
import global_resources as gr
import os

In [113]:
# Example Data process
# Read data
data_path = os.path.join(gr.default_dir, r'Data\breast-cancer-wisconsin.data')
df = gr.read_and_return_pd_df(data_path)


df.replace('?', np.nan, inplace = True)
df.dropna(inplace = True)
# df.replace('?', -99999, inplace = True)
df.drop(['id'], axis = 1, inplace = True)
df["bare_nuclei"] = df["bare_nuclei"].astype(np.int64)
df.dropna(inplace = True)


The input file path is: 
d:\Important Files\Repositories\Quantitative-Investment-Algorithms\Data\breast-cancer-wisconsin.data
Reading the input file...


## SVC

In [114]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Current device: {device.capitalize()}.")


X = np.array(df.drop(['class'], axis = 1)).astype('float64')
y = np.array(df['class']).astype('float64')

y = np.where(y == 4, 1, np.where(y == 2, -1, y))

X_gpu = torch.tensor(X, device = device)
y_gpu = torch.tensor(y, device = device)

Current device: Cuda.


In [115]:
def shuffle_tensor_row_wise(ts = X_gpu):
    indices = torch.randperm(ts.shape[0])
    ts = ts[indices]

shuffle_tensor_row_wise(X_gpu)

In [116]:
def create_random_weights_bias(shape = X_gpu.shape[-1] + 1):
    weights_bias = torch.rand(1, shape, dtype = torch.float64, device = device)
    weights_bias = torch.squeeze(weights_bias)
    weights_bias[-1] = 0
    return weights_bias

weights_bias = create_random_weights_bias()
print(weights_bias)
print(weights_bias.shape)
print(weights_bias[-1])

tensor([0.6523, 0.1372, 0.8973, 0.9162, 0.2612, 0.1198, 0.2556, 0.3937, 0.8526,
        0.0000], device='cuda:0', dtype=torch.float64)
torch.Size([10])
tensor(0., device='cuda:0', dtype=torch.float64)


In [117]:
# print(weights_bias.T)
# random_tensor = torch.rand(9, device = device, dtype = torch.float64)
# print(random_tensor)
# outcome = torch.matmul(weights_bias, random_tensor)
# print(outcome)
print(weights_bias[:-1])
print(weights_bias[-1])

tensor([0.6523, 0.1372, 0.8973, 0.9162, 0.2612, 0.1198, 0.2556, 0.3937, 0.8526],
       device='cuda:0', dtype=torch.float64)
tensor(0., device='cuda:0', dtype=torch.float64)


In [118]:
def cal_signed_distance(n_point, hyperplain_weights = weights_bias[:-1], hyperplain_bias = weights_bias[-1].item()):
    raw_scores = torch.matmul(n_point, hyperplain_weights) + hyperplain_bias
    
    weight_norm = torch.norm(hyperplain_weights)
    distances = raw_scores / weight_norm
    
    return distances

distances = cal_signed_distance(X_gpu, hyperplain_weights = weights_bias[:-1] + 1, hyperplain_bias = weights_bias[-1])
print(distances.shape)
print(y_gpu.shape)

torch.Size([683])
torch.Size([683])


In [119]:
def hinge_loss(distances = distances, labels = y_gpu, margin=1.0):
    distances = distances.squeeze()
    
    losses = torch.clamp(margin - labels * distances, min = 0)
    
    return losses.mean()

loss = hinge_loss(distances, y_gpu)
print(loss)

tensor(3.7871, device='cuda:0', dtype=torch.float64)


In [120]:
print(y_gpu.shape)

torch.Size([683])


In [121]:
def update_model(X = X_gpu, y = y_gpu, weights = weights_bias[:-1], bias = weights_bias[-1], learning_rate = 0.01):
    
    weights.requires_grad_(True)
    bias.requires_grad_(True)
    
    distances = cal_signed_distance(X, weights, bias)
    
    loss = hinge_loss(distances, y)
    
    loss.backward()
    
    with torch.no_grad():
        weights -= learning_rate * weights.grad
        bias -= learning_rate * bias.grad
    
    weights.grad.zero_()
    bias.grad.zero_()
    
    return loss.item(), weights, bias

In [156]:
# ----- Example Training Loop -----
num_epochs = 100
learning_rate = 0.01

for epoch in range(num_epochs):
    loss_value, weights, bias = update_model(X_gpu, y_gpu)
    print(f"Epoch {epoch + 1:03d} | Loss: {loss_value:.4f}")

Epoch 001 | Loss: 0.0704
Epoch 002 | Loss: 0.0704
Epoch 003 | Loss: 0.0704
Epoch 004 | Loss: 0.0704
Epoch 005 | Loss: 0.0704
Epoch 006 | Loss: 0.0704
Epoch 007 | Loss: 0.0705
Epoch 008 | Loss: 0.0704
Epoch 009 | Loss: 0.0703
Epoch 010 | Loss: 0.0704
Epoch 011 | Loss: 0.0705
Epoch 012 | Loss: 0.0708
Epoch 013 | Loss: 0.0713
Epoch 014 | Loss: 0.0712
Epoch 015 | Loss: 0.0711
Epoch 016 | Loss: 0.0706
Epoch 017 | Loss: 0.0705
Epoch 018 | Loss: 0.0704
Epoch 019 | Loss: 0.0704
Epoch 020 | Loss: 0.0703
Epoch 021 | Loss: 0.0703
Epoch 022 | Loss: 0.0705
Epoch 023 | Loss: 0.0714
Epoch 024 | Loss: 0.0711
Epoch 025 | Loss: 0.0712
Epoch 026 | Loss: 0.0708
Epoch 027 | Loss: 0.0707
Epoch 028 | Loss: 0.0704
Epoch 029 | Loss: 0.0704
Epoch 030 | Loss: 0.0704
Epoch 031 | Loss: 0.0704
Epoch 032 | Loss: 0.0704
Epoch 033 | Loss: 0.0704
Epoch 034 | Loss: 0.0703
Epoch 035 | Loss: 0.0704
Epoch 036 | Loss: 0.0715
Epoch 037 | Loss: 0.0711
Epoch 038 | Loss: 0.0712
Epoch 039 | Loss: 0.0706
Epoch 040 | Loss: 0.0704
